# Part 2: Approximate Solution Methods

- In order to be able to generalize, we must combing RL with existing reneralization methods
- The kind of generalization we require is often called function approximation beacuse it takes examples from a desired function(e.g., a value function) and attemps to generalize from them to construct an approximation of the entire function(i.e. supervised learning)
- RL with fxn approximation involves a number of new issues that do not normally arise is conventional supervised learning, such as nonstationarity, bootstrapping and delayed targets.

# Chapter 9: On-Policy Prediction with Approximation

- We will consider the use of function approximation for estimating the state-value function from on-policy data
    - i.e. approximate $v_{\pi}$ from experience generated using a known poloicy $\pi$
- The approximate value fxn is represented as a paramaterized function form with weight vector **w** $\in R^d$.
- We will write $\hat{v}(s,$**w**$) \approx v_{\pi}(s)$ for the approximate value of state s given weight vector **w**.
    - For example $\hat{v}$ might be the fxn computed by a neural network, with **w** the vector of connection weightes in all the layers.
- Typically, the number of weights (the dimensionality of **w**) is much less than the number of states, and changing one weight changes the estimated vlaue of many states.
    - Consequently, when a single state is updated, the change generalizes from that state to affect the values of many other states
    - Such generalization makes the learning potentially more powerful but also more difficult to manage and understand.
- Extending RL to fxn approximation also makes it applicable to partially observable problems, in which the full state is not available to the agent.
- If $\hat{v}$ does not allow the estimated value to depend on certain aspects of the state, then it is just as if those aspects are unobservable.

# 9.1 Value-function Approximation

- All of the prediction methods covered so far(i.e. tabular methods) have been described as updates to an estimated value function that shift its value at particular states toward a "backed-up value," or *update target*, for that state
- Notation:
    - Let $s \rightarrow u$ refer to an indivual update where *s* is the state updated and *u* is the update targer that *s*'s estimated value function is shifted toward
    - For example:
        - The MC update for value prediction is $S_{t} \rightarrow G_{t}$
        - The TD(0) update is $S_{t} = R_{t+1} + \gamma \hat{v}(S_{t+1},\textbf{w}_{t})$ 
        - The n-step TD update is $S_{t} \rightarrow G_{t:t+n}$
        - In DP, the policy evaluation update is $s \rightarrow E_{\pi}[R_{t+1} + \gamma \hat{v}(S_{t+1}, \textbf{w}_{t}) | S_{t} = s]$, where an arbitrary state s is updated, whereas in the other cases the state was encountered in actual experience.
- In a sense, the update $s \rightarrow u$ means that the estimated value for state *s* should be more like the update target *u*
- In tabular methods, the actual update to one state shifted the value of that state closer to the update target but left the values of other states unchanged.  
- Now we permit arbitrarily complex and sophisticated methods to implement the update, and updating at *s* generalizes so that the estimated values of many other states are changed as well
- ML methods that learn to mimic input-ouput examples in this way are called *supervised learning* mehtods, and then the outputs are numbers, luke *u*, the process is often called *funcition approximation*.
- Function approximation methods expect to receive examples of the desired input-output behavior of the function they are trying to approximate.
- We use these methods for value prediction by passing them $s \rightarrow g$ of each update as a training example.
- We then interpret the approximate function they produce as an estimated value function
- Viewing each update as a conventional training example enables us to use a wide range of existing function approximation methods for value prediction.
- However, not all function approximation methods are equally well suited for use in reinforcement learning.
    - In RL, it is important that learning be able to occur online, while the agent interavtes with its environment.
    - In addition, RL generally requires fxn approximation methods able to handle nonstationary target fxns

# 9.2 The Prediction Objective $(\overline{VE})$

- With genuine approximation, an update at one state affects many others, and, it is not possible to get values of all states correct.
- By assumption we have far more states than weights, so making one state's estimate more accurate invariably means making others' less accurate. We are obligated to say which states we care most about.
- We must specify a state distribution $ \mu(s) \geq 0, \sum_{s} \mu(s) = 1$, representing how much we care about the error in each state *s*
- By the error in a state *s* we mean the square of the difference between the approcimate value $\hat{v}(s, \textbf{w})$ and the true value $v_{\pi}(s)$
- By weighting the error over the state space by $\mu$, we obtain a natural objective functionm the *Mean Squared Value Error*, denoted $\overline{VE}$:
$$\overline{VE}(\textbf{w}) \doteq \sum_{s \in S} \mu(s)[v_{\pi}(s) - \hat{v}(s, \textbf{w})]^2$$
- The sqare root of $\overline{VE}$ gives a rough measure of how much the approximate values differ from teh true values and is often used in polots.
- Often $\mu(s)$ is chosen to be the fraction of time spent in *s*.
    - Under on-policy training, this is called the *on-policy distribution*
    - In continuing tasks, the on-policy distribution is the stationary distribution under $\pi$
    - In episodic taks, the on-policy distribution is different because it depends on how the initial states of episodes are chosen... look for insert on page 199 for more details
- This completes the specification of the learning objective
- Note: Remember that our ultimate purpose-the reason we are learning a value function-is to find a better policy.  The best value function for this purpose is not necessarily the best for minimizing $\overline{VE}$ so it is not clear that this is the prprt performance objective for RL but we don't have any better alternatives at this point
- An ideal goal in terms of $\overline{VE}$ would be to find a global optimum, but this is only sometimes possible for simple function approximators, like linear ones and is rarely possible for complex fxn approximators.  Thus, complex function approximators may seek to converge instead to a *local optimum*, a weight vector $\textbf{w}^{*}$ for which $\overline{VE}(\textbf{w}^{*}) \leq \overline{VE}(\textbf{w})$ for all **w** in some neighborhood $\textbf{w}^{*}$

# 9.3 Stochastic-gradient and Semi-gradient Methods

- *Stochastic gradient-descent* methods try to minimize error on observed examples by adjusting the weight vector after each example by a small amount in the direction that would reduce te error on that example:
$$ \textbf{w}_{t+1} \doteq \textbf{w}_{t} - \frac{1}{2} \alpha \nabla[v_{\pi}(S_{t}) - \hat{v}(S_{t}, \textbf{w}_{t})]^2  = \textbf{w}_t + \alpha[v_{\pi}(S_{t}) - \hat{v}(S_{t}, \textbf{w}_{t})]\nabla \hat{v}(S_{t}, \textbf{w}_{t})$$

- Gradient descent methods are stochastic when the update is done, as here, on only a single example, which might have been selected stochastically.
- Remember that we do not seek or expect to find a value function that has zero error for all states, but only an approximation that balances the errors in different states.
- The convergence results for SGD methods assume that $\alpha$ decreases over time.
- Now consider the case in which the target output, here denoted $U_{t} \in R$ of the *t*th training example, $S_{t} \rightarrow U_{t}$, is not the true value, $v_{\pi}(S_{t})$, but some approximation to it.
    - For example, $U_{t}$ might be a noise-corrupted version of $v_{\pi}(S_{t})$, or it might be one of the bootstrapping targets using $\hat{v}$
    - In these cases we cannot perform the exact update because $v_{\pi}(S_{t})$ is unknown, but we can approximate it by substituting $U_{t}$ in place for $v_{\pi}(S_{t})$.  This yields the following result:
    $$ \textbf{w}_{t+1} \doteq \textbf{w}_{t} + \alpha[U_{t} - \hat{v}(S_{t}, \textbf{w}_{t})] \nabla \hat{v}(S_{t}, \textbf{w}_{t})$$
    - If $U_{t}$ is an unbiased estimate, that is, if $E[U_{t} | S_{t} = s] = v_{\pi}(S_{t})$, for each t then $\textbf{w}_{t}$ is guaranteed to converge to a local optimum under the usual stochastic approximation conditions for decreasing $\alpha$
- The Monte Carlo target $U_{t} \doteq G_{t}$ is an unbiased estimate of $v_{\pi}(S_{t}$ and the convergence guarantees hold
- Convergence is not guaranteed if a bootstrapped estimate of $v_{\pi}(S_{t})$  is used as the target $U_{t}$
    - i.e. Bootstrapping targets such as n-step returns or the DP target are not guaranteed to converge
- One way to look at this is that convergence relies on the target being independent of $\textbf{w}_{t}$
- Thus, bootstapping methods are not instances of true gradient descent because they only take into account the effect of changing the weight vector $\textbf{w}_{t}$ on the estimate, but ignore its effect on the target.  Because of this, we call them *semi-gradient methods*
    - Although there is a lack of robust convergence in semi-gradient methods, they still offer many advantages over gradient methods
        - They typically enable significantly faster learning
        - They enable learning to be continual and onlinem without waiting for the end of an episode
- A prototypical semi-gradient method is TD(0)

# 9.4 Linear Methods

- Not going to go into a bunch of detail here so this section will be brief.  Also, skipping section 9.5
- Let $\hat{v}(.,\textbf{w})$ be a linear function of the weight vector, **w**.  Correspoing to every state *s*, there is a real-values vector $\textbf{x}(s) \doteq (x_{1}(s), x_{2}(s), ... , x_{d}(s))^{T}$, with the same number of components as **w**.
- We call **x**(s) the *feauture vector* representing state s
- Linear methods approximate the state-value function by the inner product between **w** and **x**(s): 
$$ \hat{v}(s, \textbf{w}) \doteq \textbf{w}^{T}\textbf{x}(s) \doteq \sum_{i=1}^{d} w_{i}x_{i}(s)$$
- The SGD update for the linear case is the following:
$$\textbf{w}_{t+1} \doteq \textbf{w}_{t} + \alpha[U_{t} - \hat{v}(S_{t}, \textbf{w}_{t})]\textbf{x}(S_{t})$$

# 9.6 Selecting Step-Size Parameters Manually

- Most SGD methods require the designer to select an appropriate step-size paramater $\alpha$
- The classic choice $\alpha_{t} = \frac{1}{t}$ which produces averages in tabular MC methods, is not appropriate for TD methods, for nonstationary problems or for any method using function approximation
- A good rule of thumb for setting the step-szie paramater of liner SGD metods is then:
$$ \alpha \doteq (\tau\mathbb{E}[\textbf{x}^{T}\textbf{x}])^{-1}$$ 
     - where x is a random feauture vector chosen from the same distribution as input vevtors will be in the SGD

# 9.7 Nonlinear Function Approximation: Artificial Neural Networks

- ANNs typically learn by a SGD method. Each weight is adjusted in . a diretion aimed at improving the network's overall performance as measured by an objective function to be either minimized or maximized.
- In most common supervised learning case, the objective function is the expected error, or loss, over a set of labeled training examples.
- In RL, ANNs can use TD errors to learn value functions, or they can aim to maximize expected reward as in a policy-gradient algorithm.
- Overfitting is less of a problem for online reinforcement learning that does not rely on limited training sets, but generalizing effectivelt is still an important issue.
- Advances in Deep Learning have contributed to RL although current RL theory is mostly limited to methods using tabular or linear function approximation methods.